In [1]:
import pandas as pd
from pathlib import Path
import pandas as pd
import numpy as np
from datetime import datetime
from tqdm import tqdm
from plotnine import *
import seaborn as sns
import matplotlib.pyplot as plt
from pandas.api.types import CategoricalDtype

from enhance_ocod.price_paid_process import building_gazetteer
path = '../data/processed_price_paid/price_paid_2024.parquet'
ONSPD_path = "../data/ocod_history_processed/OCOD_FULL_2017_03.parquet"
figures_folder = Path('../figures')

In [2]:
outframe = building_gazetteer()

0it [00:00, ?it/s]

31it [00:46,  1.52s/it]


In [13]:
outframe.loc[outframe['building'].str.contains('hamlyn')]

,building,oa11cd,lsoa11cd,msoa11cd,lad11cd
151365,hamlyn house,E00168183,E01033030,E02000550,E09000018
181977,hamlyn house,E00157241,E01030838,E02006433,E07000215
286525,hamlyn house,E00124070,E01024441,E02005090,E07000111
339609,hamlyn house,E00013813,E01002770,E02000557,E09000019
421166,hamlyn house,E00105180,E01020726,E02004311,E06000047
440135,hamlyn house,E00154099,E01030215,E02006298,E07000244
449988,hamlyn house,E00102313,E01020198,E02004217,E07000045


In [3]:
cleaned_ppd = pd.read_parquet(path)

cleaned_ppd

,price,date_of_transfer,property_type,paon,locality,district,ppd_category_type,street_name2,street_number,postcode2,lad11cd,oa11cd,lsoa11cd,msoa11cd
0,185000,2024-09-03 00:00,F,63a,<na>,NEWHAM,B,stamfordroad,63,e61lp,E09000025,E00017801,E01003522,E02000731
1,220000,2024-09-03 00:00,F,63a,<na>,NEWHAM,B,stamfordroad,63,e61lp,E09000025,E00017801,E01003522,E02000731
2,340000,2024-10-10 00:00,F,215,<na>,SOUTHWARK,B,lordshiplane,215,se228jf,E09000028,E00019918,E01003956,E02000836
3,545000,2024-10-30 00:00,F,8b,<na>,LEWISHAM,B,mountpleasantroad,8,se136rb,E09000023,E00016657,E01003291,E02000664
4,625000,2024-01-12 00:00,F,9,<na>,LAMBETH,B,rosedeneavenue,9,sw162ls,E09000022,E00015917,E01003152,E02000644
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
737250,300000,2024-02-02 00:00,T,11a,<na>,WEST OXFORDSHIRE,A,burfordroad,11,ox75eb,E07000181,E00146600,E01028783,E02005993
737251,640000,2024-07-11 00:00,T,73,littlemore,OXFORD,A,thecrescent,73,ox44xq,E07000178,E00145462,E01028553,E02005955
737252,600000,2024-07-19 00:00,D,highlands,blewbury,VALE OF WHITE HORSE,A,londonroad,None,ox119nx,E07000180,E00146237,E01028712,E02005992
737253,495000,2024-06-19 00:00,S,86,caulcott,CHERWELL,A,southstreet,86,ox254ne,E07000177,E00145229,E01028505,E02005930


In [26]:
test = cleaned_ppd.loc[cleaned_ppd['paon'].str.contains('wharf|building|apartment|house', na = False)].copy()
test['building']= test['paon'].str.split(',').str[0]
test.loc[:,['building', 'oa11cd', 'lsoa11cd', 'msoa11cd', 'lad11cd' ]]

,building,oa11cd,lsoa11cd,msoa11cd,lad11cd
7,koa house,E00022972,E01033132,E02000924,E09000032
8,koa house,E00022972,E01033132,E02000924,E09000032
9,koa house,E00022972,E01033132,E02000924,E09000032
14,koa house,E00022972,E01033132,E02000924,E09000032
15,clement apartments,E00176509,E01033736,E02000316,E09000011
...,...,...,...,...,...
737099,the corner house,E00146518,E01028766,E02006004,E07000181
737122,hightown house,E00144857,E01028429,E02005927,E07000177
737199,sarah siddons house,E00150441,E01029526,E02006149,E07000194
737227,essex house,E00145243,E01028508,E02005921,E07000177


In [30]:
test.drop_duplicates()

,price,date_of_transfer,property_type,paon,locality,district,ppd_category_type,street_name2,street_number,postcode2,lad11cd,oa11cd,lsoa11cd,msoa11cd,building
7,549900,2024-01-15 00:00,F,"koa house, 15",<na>,WANDSWORTH,B,electricboulevard,15,sw118da,E09000032,E00022972,E01033132,E02000924,koa house
8,554600,2024-01-15 00:00,F,"koa house, 15",<na>,WANDSWORTH,B,electricboulevard,15,sw118da,E09000032,E00022972,E01033132,E02000924,koa house
9,582500,2024-01-15 00:00,F,"koa house, 15",<na>,WANDSWORTH,B,electricboulevard,15,sw118da,E09000032,E00022972,E01033132,E02000924,koa house
14,546250,2024-01-25 00:00,F,"koa house, 15",<na>,WANDSWORTH,B,electricboulevard,15,sw118da,E09000032,E00022972,E01033132,E02000924,koa house
15,453600,2024-03-07 00:00,F,"clement apartments, 4",<na>,GREENWICH,B,brigadierwalk,4,se186zp,E09000011,E00176509,E01033736,E02000316,clement apartments
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
737099,640000,2024-06-14 00:00,D,the corner house,brize norton,WEST OXFORDSHIRE,A,elmgrove,None,ox183nf,E07000181,E00146518,E01028766,E02006004,the corner house
737122,220000,2024-06-28 00:00,F,hightown house,<na>,CHERWELL,A,hightowngardens,None,ox169gg,E07000177,E00144857,E01028429,E02005927,hightown house
737199,69000,2024-07-09 00:00,F,sarah siddons house,<na>,LICHFIELD,A,wadestreet,None,ws136hl,E07000194,E00150441,E01029526,E02006149,sarah siddons house
737227,1245000,2024-01-30 00:00,D,essex house,horley,CHERWELL,A,<na>,None,ox156bj,E07000177,E00145243,E01028508,E02005921,essex house


In [9]:
    dtype_dict = {
        "paon": "string",
        "street": "string",
        "locality": "string",
        "postcode": "string",
        "price": "int32",
        "date_of_transfer": "string",
        "district": "string",
        "property_type": CategoricalDtype(categories=["D", "S", "T", "F", "O"]),
        "ppd_category_type": CategoricalDtype(categories=["A", "B"]),
    }

    price_paid_headers = [
        "transaction_unique_identifier",
        "price",
        "date_of_transfer",
        "postcode",
        "property_type",
        "old_new",
        "duration",
        "paon",
        "saon",
        "street",
        "locality",
        "town",
        "district",
        "county",
        "ppd_category_type",
        "record_status",
    ]

In [17]:
test = pd.read_csv('/teamspace/studios/this_studio/enhance_ocod/data/price_paid_data/price_paid_complete_may_2025.csv', nrows = 10000,
            names=price_paid_headers,
            dtype=dtype_dict,)

In [21]:
test['saon'].unique()

array([nan, '1', 'FLAT 243', 'FLAT B', 'FLAT 39', 'FLAT 1', 'FLAT 2B',
       'FLAT 2', 'FLAT 7', '7', 'FLAT 6', 'THE BUNGALOW', 'FLAT 4A', '5',
       '2', 'HOPLOFT HOUSE, 1', 'FLAT 8', 'FLAT 4', '31',
       '4 BROADOAK COTTAGES', 'FLAT 3', '28', 'FLAT 21', 'FLAT 7A', '22',
       'FLAT 20', '41', 'FLAT 5', 'FLAT 11', '56', '3', '18',
       'FIRST FLOOR FLAT', 'FLAT E', '8', 'UNIT 2', 'HALL FLOOR FLAT',
       'THE COTTAGE', 'UNIT C4', 'FLAT 228', 'UPPER MAISONETTE',
       'FLAT 26', 'APARTMENT 3', 'FLAT 23', '63', 'FLAT 18', '7A',
       'FLAT 24', 'FLAT 188', '11', 'FLAT 57', '12', '9', 'APARTMENT 2',
       'FLAT 6B', 'FLAT 72', 'FLAT 112', 'FLAT 25', 'FLAT 12',
       'QUARRY HOUSE', 'FLAT 17', 'FLAT 9', 'HOLMLEA', 'FLAT 15',
       'FLAT 40', 'FLAT 44', 'GROUND FLOOR FLAT', 'UNITS 1 AND 2',
       'SECOND FLOOR FLAT', 'BASEMENT FLAT', 'FLAT 10',
       'BASEMENT AND GROUND FLOOR MAISONETTE', '6', '4', 'FLAT 37', '24',
       'SOUTH COTTAGE', 'FLAT 16', 'FLAT 254', 'PENTHOUSE F

In [33]:
test.loc[test['saon'].str.contains('UNIT', na = False)]

,transaction_unique_identifier,price,date_of_transfer,postcode,property_type,old_new,duration,paon,saon,street,locality,town,district,county,ppd_category_type,record_status
666,{85866A64-C0FA-143F-E053-6B04A8C06A15},230000,1995-05-17 00:00,WD7 8PW,D,N,F,THE FRUIT FARM,UNIT 2,COMMON LANE,<NA>,RADLETT,HERTSMERE,HERTFORDSHIRE,A,A
677,{5376B385-7833-34C1-E053-6B04A8C09FF6},84600,1995-11-24 00:00,RG28 7BP,T,N,F,DOLPHIN HOUSE,UNIT C4,PEGASUS COURT,<NA>,WHITCHURCH,BASINGSTOKE AND DEANE,HAMPSHIRE,A,A
1529,{B26F8DD5-4216-475D-B916-44F50D737398},155000,1995-08-31 00:00,<NA>,T,N,F,46,UNITS 1 AND 2,HIGH STREET,TARPORLEY,TARPORLEY,VALE ROYAL,CHESHIRE,A,A
5394,{66AAA31C-A4E7-41F7-AB6B-5473B4E11C6E},170000,1995-06-05 00:00,SE1 2BD,F,N,L,LLOYDS WHARF,UNIT B11,MILL STREET,<NA>,LONDON,SOUTHWARK,GREATER LONDON,A,A


In [30]:
test.loc[test['paon'].str.contains('FLAT', na = False)]

,transaction_unique_identifier,price,date_of_transfer,postcode,property_type,old_new,duration,paon,saon,street,locality,town,district,county,ppd_category_type,record_status
229,{AE4D86D4-3D26-4619-E053-6C04A8C03CD0},80500,1995-12-11 00:00,NW2 3UU,F,N,L,"GROUND FLOOR FLAT, 38",NaN,ANSON ROAD,<NA>,LONDON,BRENT,GREATER LONDON,A,A
2616,{DF518DAE-4157-493B-BD52-3DD2F72610CD},69950,1995-12-15 00:00,DT4 0DS,F,Y,L,FLAT 5,NaN,ST MICHAELS COURT,<NA>,WEYMOUTH,WEYMOUTH AND PORTLAND,DORSET,A,A
6977,{7968C5CC-E627-4AD6-8E61-5B864A8A24C1},51500,1995-06-16 00:00,CF10 4NG,F,N,L,FLAT 29,NaN,WINDLASS COURT,CARDIFF,CARDIFF,CARDIFF,CARDIFF,A,A
